# Types of Routing in DRF

In [ ]:
# Note: All the routings should be done in the urls.py file in your Django app.

In [ ]:
from django.contrib.sessions.backends.base import UpdateError
# Regular routes

# views.py
from django.shortcuts import render, get_object_or_404
from rest_framework.generics import CreateAPIView, ListAPIView, RetrieveAPIView, DestroyAPIView, UpdateAPIView, \
    ListCreateAPIView, RetrieveUpdateAPIView, RetrieveDestroyAPIView
from rest_framework.permissions import IsAuthenticated
from rest_framework.response import Response
from rest_framework import status
from rest_framework.decorators import api_view, permission_classes
from rest_framework.views import APIView
from rest_framework.viewsets import ReadOnlyModelViewSet


@api_view(['POST', 'GET'])
def books(request):
    return Response('List of books', status=status.HTTP_200_OK)

# urls.py
from django.urls import path
# from . import views

urlpatterns = [
    path('books/', views.books),
]


In [ ]:
# Routing to a class method

# views.py
from django.shortcuts import render
from rest_framework.response import Response
from rest_framework import status
from rest_framework.decorators import api_view

class Orders():
    @staticmethod
    @api_view()
    def listOrders(request):
        return Response({'message': 'list of orders'}, 200)

# urls.py
from django.urls import path
# from . import views

urlpatterns = [
    path('orders/', views.Orders.listOrders)
]


In [ ]:
# Routing class-based views

# views.py
from django.shortcuts import render
from rest_framework.response import Response
from rest_framework import status

class BookView(APIView):
    def get(self, request, pk):
        return Response({'message':"single book with id" + str(pk)}, status.HTTP_200_OK)
    def put(self, request, pk):
        return Response({'title': request.data.get('title')}, status.HTTP_200_OK)

# urls.py
from django.urls import path
from . import views

urlpatterns = [
    path('books/<int:pk>', views.BookView.as_view())
]

In [ ]:
# Routing classes that extend viewsets

# views.py

from django.shortcuts import render
from rest_framework.response import Response
from rest_framework import status

class BookView(viewsets.ViewSet):
    def list(self, request):
        return Response({'message':"All books"}, status.HTTP_200_OK)
    def create(self, request):
        return Response({'message':"Creating a book"}, status.HTTP_200_OK)
    def update(self, request, pk=None):
        return Response({'message':"Updating a book"}, status.HTTP_200_OK)
    def retrive(self, request, pk=None):
        return Response({'message':"Displaying a book"}, status.HTTP_200_OK)
    def partial_update(self, request, pk=None):
        return Response({'message':"Partially updating a book"}, status.HTTP_200_OK)
    def destroy(self, request, pk=None):
        return Response({'message':"Deleting a book"}, status.HTTP_200_OK)

# urls.py
from django.urls import path
from . import views

urlpatterns = [
    path('books/', views.BookView.as_view(
        {
            'get': 'list',
            'post': 'create',
        }
    )),
    path('books/<int:pk>', views.BookView.as_view(
        {
            'get': 'retrive',
            'put': 'update',
            'patch': 'partial_update',
            'delete': 'destroy',
        }
    )),
]

In [ ]:
# Routing with SimpleRouter class in DRF

# If you have a class that extends ViewSets then you can use different types of built-in routers to map those classes in your urls.py file.
#  Doing things this way means you don’t have to map the individual methods as you did in the previous section.

# urls.py
from rest_framework.routers import SimpleRouter

router = SimpleRouter(trailing_slash=False)
router.register('books', views.BookView, basename='books')
urlpatterns = router.urls

# After mapping, you can access the api/books and api/books/1 endpoints with the same methods as in the previous example.
# Did you notice that the argument trailing_slash=False was passed, instantiating the SimpleRouter object? Without this argument, your API endpoints will have a trailing slash. And, since you don’t want a trailing slash at the end of your API endpoints, you have to pass this argument.

In [ ]:
# Routing with DefaultRouter class in DRF
# There is another type of router called DefaultRouter which provides an extra benefit over the SimpleRouter.
# It creates an API root endpoint with a trailing slash that displays all your API endpoints in one place.

# urls.py
from rest_framework.routers import DefaultRouter
router = DefaultRouter(trailing_slash=False)
router.register('books', views.BookView, basename='books')
urlpatterns = router.urls

# Again, after mapping, you can access the api/books and api/books/1 endpoints with the same methods as in the previous examples.

# Generic views and ViewSets in DRF

In [ ]:
# DRF comes with many generic views and ViewSet to speed up API development.
# ViewSets are simple class-based views, but they come with benefits.
# There are a few ViewSets classes available in DRF that you can use to quickly scaffold a functioning API CRUD project.

from rest_framework import viewsets

# viewsets.ViewSet methods
# list()              GET     Display resource collection
# create()            POST    Create new resource
# retrive()           GET     Display a single resource
# update()            PUT     Completely replace a single resource with new data
# partial_update()    PATCH   Partially update a single resource
# destroy()           DELETE  Delete a single resource


In [ ]:
# To perform DB operation automatically there are 2 more viewsets
# 1, ModelViewSet (viewsets.ModelViewSet)
#     ->  it can automatically handle CRUD operations
#     ->  All you must do is give this class a queryset and a serializer, and everything else will be done automatically.

# 2. ReadOnlyModelViewSet (viewsets.ReadOnlyModelViewSet)
#     ->  it can only display a single resource and resource collection.
#     ->  No write-operation is allowed by such views, so it doesn’t handle POST, PUT, PATCH or DELETE methods.

In [ ]:
# Generic Views
# Generic views are another way of quickly writing class-based views to scaffold fully functional CRUD API projects.

from rest_framework import generics

# All generic view classes require a queryset and a serializer to work properly.

# Generic view classes:

# CreateAPIView               POST                  Create new resource
# ListAPIView                 GET                   Display resource collection
# RetrieveAPIView             GET                   Display a single resource
# DestroyAPIView              DELETE                Delete a single resource
# UpdateAPIView               PUT and               PATCH Replace or partially update a single resource
# ListCreateAPIView           GET,POST              Display resource collection and create a new resource
# RetrieveUpdateAPIView       GET,PUT,PATCH         Display a single resource and replace or partially update it
# RetrieveDestroyAPIView      GET,DELETE            Display a single resource and delete it
# RetriveUpdateDestroyAPIView GET,PUT,PATCH,DELETE  Display, replace or update and delete a single resource

class MenuItemView (generics.ListAPIView, generics.CreateAPIView) # or
class MenuItemView (generics.ListCreateAPIView)
# These both do the same job

In [ ]:
# Authentication and selective authentication

# If you want all API calls to be authenticated in a class-based view that extends the generic views, you can add the permission_classes public attribute in the class.

# If you want to selectively enable authentication for some calls, like POST, PUT, PATCH and DELETE then you need to override the get_permission method in your class-based view like this.

def get_permissions(self):
    permission_classes = []
    if self.request.method != 'GET':
        permission_classes = [IsAuthenticated]
    return [permission() for permission in permission_classes]

# This way, anyone will be able to make GET call, but other HTTP methods like POST, PUT, PATCH and DELETE will require authentication or a valid user token


In [ ]:
# Return items for the authenticated user only

class OrderView(generics.ListCreateAPIView):
    queryset = Order.objects,all()
    serializer_class = OrderSerializer
    permission_classes = [IsAuthenticated]

    def get_queryset(self):
        return Order.objects.all().filter(user=self.request.user)

# Override default behavior

class OrderView(generics.ListCreateAPIView):
    queryset = Order.objects.all()
    serializer_class = OrderSerializer

    def get(self, request,*args, **lwargs):
        return Response("New response")

# Serializers

In [ ]:
# Serialize model relationships

# serializer.py
from rest_framework import serializers
from .models import MenuItem, Category

class CategorySerializer(serializers.ModelSerializer):
    class Meta:
        model = Category
        fields = ['id', 'slug', 'title']

class MenuItemSerializer(serializers.ModelSerializer):
    stock = serializers.IntegerField(source='inventory')
    price_after_tax = serializers.SerializerMethodField(method_name= 'calculate_tax')
    category = CategorySerializer()

    class Meta:
        model = MenuItem
        fields = ['id', 'title', 'price', 'stock', 'price_after_tax', 'category']
        # depth = 1

        def calculate_tax(self, product:MenuItem):
            return product.price*Decimal(1.1)

In [ ]:
# Display a related model fields field as a hyperlink

# -> In DRF you can display every related model field as a hyperlink in the API output. Like this:  http://127.0.0.1:8000/api/category/{categoryId}
# Method 1: HyperlinkedRelatedField
# views.py
from .models import Category
from .serializers import CategorySerializer

def category_detail(request, pk):
    category = get_object_or_404(Category,pk=pk)
    serialized_category = CategorySerializer(category)
    return Response(serialized_category.data)

# urls.py
path('category/<int:pk>', views/category_detail, name='category-detail')

# Tip: There is a convention you must follow when you create this view name. The rule is that you have to add -detail after the related field name, which is category in the MenuItemSerializer. This is why the view name was category-detail in this code. If the related field name was user, the view name would be user-detail.

# serializer.py
from .model import Category

class MenuItemSerializer(serializers.ModelSerializer):
    category = serializers.HyperlinkedRelatedField(
        queryset= Category.objects.all(),
        view_name= 'category-detail'
    )

    class Meta:
        model = MenuItem
        fields = ['id','title','price','inventory','category']

# menu_items
serialized_item = MenuItemSerializer(items, many=True, context={'request': request})
# Note: The argument context={'request': request} lets the menu-items endpoint display the category field as a hyperlink.

# Method 2: HyperlinkedModelSerializer

class MenuItemSerializer(serializers.HyperlinkedModelSerializer):
    class Meta:
        model = MenuItem
        fields = ['id','title','price','inventory','category']

urlpatterns = [
        path('menu-items',views.menu_items),
        path('menu-items/<int:id>',views.single_item),
        path('category/<int:pk>',views.category_detail, name='category-detail')
]